In [98]:
import numpy as np
import pandas as pd

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [100]:
class ANN:
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size, lr):
        self.input_size = input_size
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.output_size = output_size
        self.lr = lr

        self.Wij = np.random.uniform(-0.5, 0.5, (self.input_size, self.hidden_size_1)) #between input and hidden 1 layer
        self.Vjk = np.random.uniform(-0.5, 0.5, (self.hidden_size_1, self.hidden_size_2))#between hidden 1 and hidden 2 layer
        self.Ukl = np.random.uniform(-0.5, 0.5, (self.hidden_size_2, self.output_size))#between hidden 2 and output layer

        self.bias_j = np.random.uniform(-0.5, 0.5, (1,self.hidden_size_1)) # for hidden 1 layer
        self.bias_k = np.random.uniform(-0.5, 0.5, (1,self.hidden_size_2)) # for hidden 2 layer
        self.bias_l = np.random.uniform(-0.5, 0.5, (1,self.output_size)) # for output layer

    def sigmoid(self, x):
        return 1/(1+np.exp(-x))

    def sigmoid_derivate(self, x):
        return x*(1-x)

    def forward_propagate(self, x):
        self.hidden_1_input = np.dot(x, self.Wij)+self.bias_j
        self.hidden_1_output = self.sigmoid(self.hidden_1_input)

        self.hidden_2_input = np.dot(self.hidden_1_output, self.Vjk)+self.bias_k
        self.hidden_2_output = self.sigmoid(self.hidden_2_input)

        self.output_input = np.dot(self.hidden_2_output, self.Ukl)+self.bias_l
        self.output_output = self.sigmoid(self.output_input)

        return self.output_output

    def back_propagate(self, x, y, out):
        error = y-out
        delta_out = error * self.sigmoid_derivate(out)

        hidden_2_error = delta_out.dot(self.Ukl.T)
        hidden_2_delta = hidden_2_error * self.sigmoid_derivate(self.hidden_2_output)

        hidden_1_error = hidden_2_delta.dot(self.Vjk.T)
        hidden_2_delta = hidden_1_error * self.sigmoid_derivate(self.hidden_1_output)

        self.Ukl += self.hidden_2_output.T.dot(delta_out)*self.lr
        self.bias_l += np.sum(delta_out)*self.lr

        self.Vjk += self.hidden_1_output.T.dot(hidden_2_delta)*self.lr
        self.bias_k += np.sum(hidden_2_delta)*self.lr

        self.Wij += x.T.dot(hidden_2_delta)*self.lr
        self.bias_j += np.sum(hidden_2_delta)*self.lr

    def train(self, x, y, epochs):
        for epoch in range(epochs):
            out = self.forward_propagate(x)
            self.back_propagate(x, y, out)
            if epoch%1000 == 0:
                print(f"Epochs {epoch} Loss {np.mean(np.abs(y - out))}")

In [101]:
df = pd.read_csv("Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [102]:
df["Species"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [103]:
df["Species"] = df["Species"].replace({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2})

In [104]:
arr1 = np.array([df['SepalLengthCm'], df['SepalWidthCm'], df['PetalLengthCm'], df['PetalWidthCm']]).T
arr2 = np.array([df['Species']]).T

In [105]:
X_train, X_test, y_train, y_test = train_test_split(arr1, arr2, test_size=0.3, random_state=10)

In [106]:
encoder = OneHotEncoder()

In [107]:
y_train_hot = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()

In [108]:
input_size = X_train.shape[1]
hidden_1_size = 50
hidden_2_size = 50
output_size = y_train_hot.shape[1]
lr = 0.1
epochs = 10000

obj_ANN = ANN(input_size, hidden_1_size, hidden_2_size, output_size, lr)
obj_ANN.train(X_train, y_train_hot, epochs)

Epochs 0 Loss 0.6210261840612572
Epochs 1000 Loss 0.3333461227475785
Epochs 2000 Loss 0.33339639996520515
Epochs 3000 Loss 0.04556870997925581
Epochs 4000 Loss 0.03612998113976855
Epochs 5000 Loss 0.03612105409931306
Epochs 6000 Loss 0.025896131722761825
Epochs 7000 Loss 0.049888706831308936
Epochs 8000 Loss 0.025275932455723533
Epochs 9000 Loss 0.024868793302730906


In [109]:
print("SL|SW|PL|PW \t\t Predicted Class \t\t Actual Class")
for i in range(len(X_test)):
    predict = obj_ANN.forward_propagate(X_test[i])
    predicted_class = np.argmax(predict)
    actual = y_test[i]
    print(f"{X_test[i]}\t\t{predicted_class}\t\t{actual}")

SL|SW|PL|PW 		 Predicted Class 		 Actual Class
[6.3 2.3 4.4 1.3]		1		[1]
[6.4 2.7 5.3 1.9]		2		[2]
[5.4 3.7 1.5 0.2]		0		[0]
[6.1 3.  4.6 1.4]		1		[1]
[5.  3.3 1.4 0.2]		0		[0]
[5.  2.  3.5 1. ]		1		[1]
[6.3 2.5 4.9 1.5]		2		[1]
[5.8 2.7 4.1 1. ]		1		[1]
[5.1 3.4 1.5 0.2]		0		[0]
[5.7 2.8 4.5 1.3]		1		[1]
[5.6 3.  4.5 1.5]		1		[1]
[5.8 2.7 5.1 1.9]		2		[2]
[5.5 2.3 4.  1.3]		1		[1]
[4.9 3.  1.4 0.2]		0		[0]
[5.1 3.8 1.5 0.3]		0		[0]
[6.8 3.  5.5 2.1]		2		[2]
[6.  3.4 4.5 1.6]		1		[1]
[4.4 3.  1.3 0.2]		0		[0]
[5.1 3.7 1.5 0.4]		0		[0]
[5.  3.2 1.2 0.2]		0		[0]
[7.1 3.  5.9 2.1]		2		[2]
[6.4 2.8 5.6 2.2]		2		[2]
[6.2 2.8 4.8 1.8]		2		[2]
[4.8 3.4 1.9 0.2]		0		[0]
[5.9 3.  4.2 1.5]		1		[1]
[4.7 3.2 1.3 0.2]		0		[0]
[5.7 3.  4.2 1.2]		1		[1]
[5.5 2.6 4.4 1.2]		1		[1]
[6.8 2.8 4.8 1.4]		1		[1]
[7.7 3.8 6.7 2.2]		2		[2]
[6.6 2.9 4.6 1.3]		1		[1]
[6.2 2.9 4.3 1.3]		1		[1]
[7.2 3.  5.8 1.6]		2		[2]
[5.8 2.8 5.1 2.4]		2		[2]
[6.3 2.5 5.  1.9]		2		[2]
[4.6 3.2 1.4 0.2]		0		[0]
[6.7 3.3 5.7 2.1]

In [110]:
count = 0
total = len(X_test)
for i in range(total):
    predict = obj_ANN.forward_propagate(X_test[i])
    predicted_class = np.argmax(predict)
    actual = y_test[i]
    if predicted_class == actual:
        count+=1
accuracy = (count/total)*100
print(f"{accuracy:.2f}%")

97.78%
